In [21]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

In [22]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [23]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [24]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [25]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [26]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.
Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [27]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [28]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.inception_v3.preprocess_input

In [29]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [30]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [31]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.InceptionV3(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [32]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_2, False
Layer: conv2d_94, False
Layer: batch_normalization_99, False
Layer: activation_98, False
Layer: conv2d_95, False
Layer: batch_normalization_100, False
Layer: activation_99, False
Layer: conv2d_96, False
Layer: batch_normalization_101, False
Layer: activation_100, False
Layer: max_pooling2d_4, False
Layer: conv2d_97, False
Layer: batch_normalization_102, False
Layer: activation_101, False
Layer: conv2d_98, False
Layer: batch_normalization_103, False
Layer: activation_102, False
Layer: max_pooling2d_5, False
Layer: conv2d_102, False
Layer: batch_normalization_107, False
Layer: activation_106, False
Layer: conv2d_100, False
Layer: conv2d_103, False
Layer: batch_normalization_105, False
Layer: batch_normalization_108, False
Layer: activation_104, False
Layer: activation_107, False
Layer: average_pooling2d_9, False
Layer: conv2d_99, False
Layer: conv2d_101, False
Layer: conv2d_104, False
Layer: conv2d_105, False
Layer: batch_normalization_104, False
Layer: batch_normal

In [33]:
len(base_model.layers)

311

In [34]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [35]:
aug_layer = utils.return_data_aug_layer_for_eff_net()

In [36]:
from keras.src.regularizers import l2
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
#x = aug_layer(inputs)
x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(x, training = False)
x = layers.GlobalAvgPool2D()(x)
x = BatchNormalization()(x)

x = Dense(512, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = Dense(256, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)


x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'Inceptionv')

model_1.summary()

Model: "Inceptionv"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_193 (B  (None, 2048)              8192      
 atchNormalization)                                              
                                                                 
 dense_5 (Dense)             (None, 512)               1

In [37]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [38]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=3) 

In [39]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [40]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 15, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/15
73/74 [============================>.] - ETA: 0s - loss: 11.6755 - accuracy: 0.5428
Epoch 1: val_loss improved from inf to 8.28978, saving model to Trained_Models/Inceptionv
74/74 [==============================] - 18s 208ms/step - loss: 11.6386 - accuracy: 0.5426 - val_loss: 8.2898 - val_accuracy: 0.8487 - lr: 0.0010
Epoch 2/15
74/74 [==============================] - ETA: 0s - loss: 6.2162 - accuracy: 0.6823
Epoch 2: val_loss improved from 8.28978 to 4.69458, saving model to Trained_Models/Inceptionv
74/74 [==============================] - 16s 210ms/step - loss: 6.2162 - accuracy: 0.6823 - val_loss: 4.6946 - val_accuracy: 0.8487 - lr: 0.0010
Epoch 3/15
72/74 [============================>.] - ETA: 0s - loss: 3.8516 - accuracy: 0.7140
Epoch 3: val_loss improved from 4.69458 to 3.07008, saving model to Trained_Models/Inceptionv
74/74 [==============================] - 16s 207ms/step - loss: 3.8341 - accuracy: 0.7141 - val_loss: 3.0701 - val_accuracy: 0.8487 - lr: 0.0010
Epo

In [41]:
start_epoch = 15
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+15,initial_epoch=start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 16/30
74/74 [==============================] - ETA: 0s - loss: 0.7897 - accuracy: 0.8247
Epoch 16: val_loss improved from 0.95585 to 0.65949, saving model to Trained_Models/Inceptionv
74/74 [==============================] - 16s 212ms/step - loss: 0.7897 - accuracy: 0.8247 - val_loss: 0.6595 - val_accuracy: 0.9189 - lr: 3.0000e-04
Epoch 17/30
72/74 [============================>.] - ETA: 0s - loss: 0.7330 - accuracy: 0.8255
Epoch 17: val_loss did not improve from 0.65949
74/74 [==============================] - 3s 42ms/step - loss: 0.7351 - accuracy: 0.8263 - val_loss: 0.6689 - val_accuracy: 0.9060 - lr: 3.0000e-04
Epoch 18/30
72/74 [============================>.] - ETA: 0s - loss: 0.7068 - accuracy: 0.8303
Epoch 18: val_loss improved from 0.65949 to 0.63536, saving model to Trained_Models/Inceptionv
74/74 [==============================] - 16s 208ms/step - loss: 0.7042 - accuracy: 0.8310 - val_loss: 0.6354 - val_accuracy: 0.8932 - lr: 3.0000e-04
Epoch 19/30
72/74 [=============

In [42]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 51ms/step - loss: 0.5858 - accuracy: 0.8355


[0.5858327150344849, 0.8355029821395874]

In [43]:
base_model.trainable = True

In [44]:
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [45]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [46]:
start_epoch = 30
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Inceptionv"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_193 (B  (None, 2048)              8192      
 atchNormalization)                                              
                                                                 
 dense_5 (Dense)             (None, 512)               1

In [47]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 31/40
74/74 [==============================] - ETA: 0s - loss: 0.8658 - accuracy: 0.7281
Epoch 31: val_loss did not improve from 0.44336
74/74 [==============================] - 8s 44ms/step - loss: 0.8658 - accuracy: 0.7281 - val_loss: 4.9273 - val_accuracy: 0.1246 - lr: 3.0000e-04
Epoch 32/40
74/74 [==============================] - ETA: 0s - loss: 0.6943 - accuracy: 0.7980
Epoch 32: val_loss did not improve from 0.44336
74/74 [==============================] - 3s 42ms/step - loss: 0.6943 - accuracy: 0.7980 - val_loss: 0.5453 - val_accuracy: 0.8576 - lr: 3.0000e-04
Epoch 33/40
72/74 [============================>.] - ETA: 0s - loss: 0.6229 - accuracy: 0.8203
Epoch 33: val_loss did not improve from 0.44336
74/74 [==============================] - 3s 43ms/step - loss: 0.6252 - accuracy: 0.8200 - val_loss: 1.4857 - val_accuracy: 0.2285 - lr: 3.0000e-04
Epoch 34/40
72/74 [============================>.] - ETA: 0s - loss: 0.6058 - accuracy: 0.8234
Epoch 34: val_loss did not improve 

In [48]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 24ms/step - loss: 0.6225 - accuracy: 0.8201


[0.6224556565284729, 0.8201183676719666]

In [49]:
start_epoch = 40
for layer in base_model.layers[-75:]:
    layer.trainable = True

In [50]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Inceptionv"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_193 (B  (None, 2048)              8192      
 atchNormalization)                                              
                                                                 
 dense_5 (Dense)             (None, 512)               1

In [52]:
start_epoch = 50
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 51/60
73/74 [============================>.] - ETA: 0s - loss: 0.4268 - accuracy: 0.8750
Epoch 51: val_loss did not improve from 0.35169
74/74 [==============================] - 4s 47ms/step - loss: 0.4266 - accuracy: 0.8755 - val_loss: 7.3225 - val_accuracy: 0.0425 - lr: 9.0000e-05
Epoch 52/60
74/74 [==============================] - ETA: 0s - loss: 0.4139 - accuracy: 0.8734
Epoch 52: val_loss did not improve from 0.35169
74/74 [==============================] - 4s 43ms/step - loss: 0.4139 - accuracy: 0.8734 - val_loss: 16.1631 - val_accuracy: 0.0000e+00 - lr: 9.0000e-05
Epoch 53/60
74/74 [==============================] - ETA: 0s - loss: 0.4175 - accuracy: 0.8776
Epoch 53: val_loss did not improve from 0.35169
74/74 [==============================] - 4s 43ms/step - loss: 0.4175 - accuracy: 0.8776 - val_loss: 1.5933 - val_accuracy: 0.6222 - lr: 9.0000e-05
Epoch 54/60
74/74 [==============================] - ETA: 0s - loss: 0.3940 - accuracy: 0.8793
Epoch 54: val_loss did not imp

In [53]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 26ms/step - loss: 0.9410 - accuracy: 0.7290


[0.9409737586975098, 0.7289940714836121]

In [55]:
start_epoch = 60

for layer in base_model.layers[-125:]:
    layer.trainable = True

In [56]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000075), metrics = ['accuracy'])
model_1.summary()

Model: "Inceptionv"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_193 (B  (None, 2048)              8192      
 atchNormalization)                                              
                                                                 
 dense_5 (Dense)             (None, 512)               1

In [57]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 61/70
74/74 [==============================] - ETA: 0s - loss: 0.5859 - accuracy: 0.8094
Epoch 61: val_loss did not improve from 0.35169
74/74 [==============================] - 12s 57ms/step - loss: 0.5859 - accuracy: 0.8094 - val_loss: 48.6632 - val_accuracy: 0.0000e+00 - lr: 7.5000e-05
Epoch 62/70
74/74 [==============================] - ETA: 0s - loss: 0.4867 - accuracy: 0.8234
Epoch 62: val_loss did not improve from 0.35169
74/74 [==============================] - 4s 48ms/step - loss: 0.4867 - accuracy: 0.8234 - val_loss: 34.9214 - val_accuracy: 0.0000e+00 - lr: 7.5000e-05
Epoch 63/70
73/74 [============================>.] - ETA: 0s - loss: 0.4958 - accuracy: 0.8309
Epoch 63: val_loss did not improve from 0.35169
74/74 [==============================] - 4s 49ms/step - loss: 0.4962 - accuracy: 0.8306 - val_loss: 102.3690 - val_accuracy: 0.0000e+00 - lr: 7.5000e-05
Epoch 64/70
74/74 [==============================] - ETA: 0s - loss: 0.5062 - accuracy: 0.8217
Epoch 64: val_loss

KeyboardInterrupt: 

In [36]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 35ms/step - loss: 0.4816 - accuracy: 0.8828


[0.48157748579978943, 0.8828402161598206]

In [37]:
start_epoch = 45
for layer in base_model.layers[-300:]:
    layer.trainable = True

In [38]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000025), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [39]:
history_500 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 46/55
74/74 [==============================] - ETA: 0s - loss: 0.1190 - accuracy: 0.9856
Epoch 46: val_loss did not improve from 0.16067
74/74 [==============================] - 36s 107ms/step - loss: 0.1190 - accuracy: 0.9856 - val_loss: 0.2369 - val_accuracy: 0.9585 - lr: 2.5000e-05
Epoch 47/55
74/74 [==============================] - ETA: 0s - loss: 0.1114 - accuracy: 0.9894
Epoch 47: val_loss did not improve from 0.16067
74/74 [==============================] - 6s 83ms/step - loss: 0.1114 - accuracy: 0.9894 - val_loss: 0.4673 - val_accuracy: 0.8724 - lr: 2.5000e-05
Epoch 48/55
74/74 [==============================] - ETA: 0s - loss: 0.1195 - accuracy: 0.9886
Epoch 48: val_loss did not improve from 0.16067
74/74 [==============================] - 6s 83ms/step - loss: 0.1195 - accuracy: 0.9886 - val_loss: 0.2321 - val_accuracy: 0.9525 - lr: 2.5000e-05
Epoch 49/55
74/74 [==============================] - ETA: 0s - loss: 0.0976 - accuracy: 0.9936
Epoch 49: val_loss did not improv

In [40]:
start_epoch = 55
for layer in base_model.layers[-700:]:
    layer.trainable = True

In [41]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000025), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [42]:
history_500 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 56/65
74/74 [==============================] - ETA: 0s - loss: 0.1236 - accuracy: 0.9814
Epoch 56: val_loss did not improve from 0.16067
74/74 [==============================] - 55s 148ms/step - loss: 0.1236 - accuracy: 0.9814 - val_loss: 0.3335 - val_accuracy: 0.9308 - lr: 2.5000e-05
Epoch 57/65
74/74 [==============================] - ETA: 0s - loss: 0.1372 - accuracy: 0.9809
Epoch 57: val_loss did not improve from 0.16067
74/74 [==============================] - 8s 107ms/step - loss: 0.1372 - accuracy: 0.9809 - val_loss: 0.1625 - val_accuracy: 0.9763 - lr: 2.5000e-05
Epoch 58/65
74/74 [==============================] - ETA: 0s - loss: 0.1013 - accuracy: 0.9903
Epoch 58: val_loss did not improve from 0.16067
74/74 [==============================] - 8s 108ms/step - loss: 0.1013 - accuracy: 0.9903 - val_loss: 0.2233 - val_accuracy: 0.9594 - lr: 2.5000e-05
Epoch 59/65
74/74 [==============================] - ETA: 0s - loss: 0.0955 - accuracy: 0.9911
Epoch 59: val_loss did not impr

In [43]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 1s 35ms/step - loss: 0.4736 - accuracy: 0.8923


[0.47362667322158813, 0.892307698726654]

In [38]:
resnet = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet")


OSError: No file or directory found at /home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet

In [ ]:
resnet.evaluate(val_datagen)